# Functions

In [ ]:
!pip install --quiet --disable-pip-version-check \
    cmocean==4.0.3 \
    folium==0.20.0 \
    geopandas==1.0.1 \
    matplotlib==3.8.4 \
    nbconvert==7.17.0 \
    pandas==2.1.4 \
    requests==2.32.5 \
    shapely==2.1.2 

In [ ]:
# Load json to parse JSON text for functions module
import json

# Load sys so Python can locate files in list of directories
import sys

# Load os to get the current directory path
import os

# load inspect to examine Python objects
import inspect

# Load geopandas library to work with geoJSON files.
import geopandas as gpd

# Load pandas library for data manipulation and analysis.
import pandas as pd

# Load matplotlib library for data visualization.
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Load folium to visualize geospatial data.
import folium

# Load cmocean for colormaps.
import cmocean.cm as cmo

# Load datetime to manipulate dates and times.
from datetime import datetime

# Load requests to conduct HTTP requests to an API.
import requests

# Load time so we can time and optimize queries.
import time

# Load Point to use in coordinate mapping.
from shapely.geometry import Point

In [ ]:
#______________________________________________________________________________

def summarize_dataframe(df):
    """
    Summarizes a DataFrame/GeoDataFrame with column-level statistics.
    Args:
        df: a pandas DataFrame or geopandas GeoDataFrame to summarize
    Returns:
        DataFrame with summary statistics for each column:
            Type: column data type
            Missing: count of missing values
            Missing %: percent of missing values
            Unique: count of unique values
            min, max, mean: minimum, maximum, and mean (numeric columns only)
    Raises:
        AttributeError: If df input is not a DataFrame-like object
        ZeroDivisionError: If DataFrame is empty
    """
    summary = {
        col: {
            'Type': df[col].dtype,
            'Missing': df[col].isnull().sum(),
            'Missing %': round(df[col].isnull().sum() / len(df) * 100, 2),
            'Unique': df[col].nunique()
        }
        for col in df.columns
    }
    
    # Transpose summary so each row represents one column from df.
    summary_df = pd.DataFrame(summary).T

    # Grab numeric columns only (geometry is excluded automatically).
    numeric_cols = df.select_dtypes(include='number')

    # Compute summary statistics for numeric columns.
    if not numeric_cols.empty:
        summary_stats = numeric_cols.agg(['min', 'max', 'mean']).T.round(2)
        summary_df = summary_df.join(summary_stats)

    return summary_df

#______________________________________________________________________________

In [ ]:
#______________________________________________________________________________

def group_and_sum(df, column):
    """
    Groups a DataFrame/GeoDataFrame by a specified column,
    and sums the numeric columns.
    Args:
        df: a pandas DataFrame or geopandas GeoDataFrame
        column: Column name to group by (string)
    Returns:
        pandas DataFrame with grouped data and summed numeric values
    Raises:
        ValueError: If df contains no numeric columns to sum
    """

    # Check for numeric columns before grouping.
    numeric_cols = df.select_dtypes(include='number').columns
    numeric_cols = [col for col in numeric_cols if col != column]

    if len(numeric_cols) == 0:
        raise ValueError("DataFrame contains no numeric columns to sum")

    return df.groupby(column).sum()

#______________________________________________________________________________

In [ ]:
#______________________________________________________________________________

def min_max(df, column):
    """
    Computes the minimum and maximum values of a column.
    Args:
        df: a pandas DataFrame
        column: Column name to compute min/max for (string)
    Returns:
        (min_value, max_value) for the specified column (tuple)
    Raises:
        KeyError: If the column does not exist in the DataFrame.
    """
    return df[column].min(), df[column].max()
    
#______________________________________________________________________________

In [ ]:
#______________________________________________________________________________

def df_to_geodataframe(df, lon_col, lat_col, crs = "EPSG:4326"):
    """
    Convert a pandas DataFrame with lon/lat columns into a GeoDataFrame.
    Ensures longitude and latitude columns to numeric values, drops
    rows with missing coordinates, and creates Point geometry objects.
    Args:
        df: pandas DataFrame
        lon_col: Longitude column name (string)
        lat_col: Latitude column name (string)
        crs: Coordinate reference system (string, default: "EPSG:4326")
    Returns:
        geopandas.GeoDataFrame with Points in specified CRS
    """

    df = df.copy()

    # Ensure latitude and longitude columns are numeric.
    df[lat_col] = pd.to_numeric(df[lat_col], errors = "coerce")
    df[lon_col] = pd.to_numeric(df[lon_col], errors = "coerce")

    # Drop rows with missing data.
    df = df.dropna(subset = [lon_col, lat_col])

    # Create geometry object using Shapely Point.
    df["geometry"] = [Point(xy) for xy in zip(df[lon_col], df[lat_col])]

    # Convert DataFrame to a GeoDataFrame.
    gdf = gpd.GeoDataFrame(df, geometry="geometry", crs=crs)

    return gdf

#______________________________________________________________________________

In [ ]:
# original version of create_map
#______________________________________________________________________________

def create_map_orig(gdf, fields = None, title = None, color_code = None):
    """
    Create an interactive folium map.
    Centers the map of the gdf's center and handles coordinate 
    transformations. Optionally color-codes features by a specified column.
    Args:
        gdf: GeoDataFrame to visualize
        fields: field names to display in the tooltip (list, default = None)
        title: Name of map (string, default = None)
        color_code: Column name for color coding features (string, 
        default = None)
    Returns:
        folium.Map object with the GeoDataFrame layer added
    """
#______________________________________________________________________________

    # Make a copy to avoid modifying the original GeoDataFrame.
    gdf = gdf.copy()
    
    # Convert datetime columns to strings for JSON serialization.
    datetime_cols = gdf.select_dtypes(
        include = ['datetime64', 'datetime']
        ).columns
    for col in datetime_cols:
        gdf[col] = gdf[col].astype(str)

    # Calculate center based on CRS.
    # If already in geographic crs, then
    # project crs temporarily for centroid calculation.
    if gdf.crs and gdf.crs.is_geographic:
        # Use the UTM zone 17N for Detroit area.
        gdf_projected = gdf.to_crs(epsg = 32617)
        # Calculate center and project back to geographic.
        center = gdf_projected.geometry.centroid.to_crs(epsg = 4326)
    else:
        center = gdf.geometry.centroid.to_crs(epsg = 4326)

    # Calculate center
    lat = center.y.mean()
    lon = center.x.mean()

    # Convert the crs to latitude/longitude for folium mapping.
    gdf = gdf.to_crs(epsg = 4326)

    # Create a centered base map with zoom level of 11.
    map = folium.Map(
        location = [lat, lon],
        zoom_start = 11
        )

    # Configure styling based on color_code parameter.
    if color_code:
        # Get unique values and generate colors dynamically.
        unique_values = gdf[color_code].unique()
        num_colors = len(unique_values)
        # Choose colormap based on number of categories.
        if num_colors <= 10:
            cmap = plt.cm.get_cmap('tab10')
        elif num_colors <= 20:
            cmap = plt.cm.get_cmap('tab20')
        else:
            cmap = plt.cm.get_cmap('hsv')
        # Generate colors.
        colors = [mcolors.rgb2hex(
            cmap(i / num_colors)
            ) 
            for i in range(num_colors)
            ]
        # Create color map dictionary.
        color_map = {val: colors[i] for i, val in enumerate(unique_values)}
        # Define style function with color coding.
        style_function = lambda x: {
            "fillColor": color_map.get(x['properties'][color_code], 'gray'),
            "color": "black",
            "weight": 0.5,
            "fillOpacity": 0.7
        }

    # If color code is not specified, use one color.  
    else:
        # Define style function with single color.
        style_function = lambda x: {
            "fillColor": "blue",
            "color": "black",
            "weight": 0.5,
            "fillOpacity": 0.5
        }
    
    # Add GeoJSON layer to map.
    folium.GeoJson(
        gdf,
        name = title,
        style_function = style_function,
        tooltip = folium.GeoJsonTooltip(
            fields = fields
            ) 
            if fields 
            else None
    ).add_to(map)
    
    return map

#______________________________________________________________________________

In [ ]:
# New version of create_map function.
# Uses a muted color scheme for visibility.
#______________________________________________________________________________
def create_map(gdf, fields = None, title = None, color_code = None):
    """
    Create an interactive folium map from a GeoDataFrame.
    Centers the map on the GeoDataFrame's centroid and uses a muted 
    cmocean color scheme for better visibility.
    Args:
        gdf: geopandas.GeoDataFrame to visualize
        fields: Field names to display in tooltip (list, default = None)
        title: Name of the map layer (string, default = None)
        color_code: column name for color coding features 
        (string, default = None)
    Returns:
        folium.Map object with the GeoDataFrame layer added
    """

    gdf = gdf.copy()
    
    # Convert datetime columns to strings for JSON serialization
    datetime_cols = gdf.select_dtypes(
        include = ['datetime64', 'datetime']
        ).columns
    for col in datetime_cols:
        gdf[col] = gdf[col].astype(str)

    # Calculate center based on CRS
    # If already in geographic CRS, project temporarily for accurate centroid
    if gdf.crs and gdf.crs.is_geographic:
        # Use UTM zone 17N for Detroit area
        gdf_projected = gdf.to_crs(epsg = 32617)
        # Project back to EPSG:4326 immediately
        center = gdf_projected.geometry.centroid.to_crs(epsg = 4326)
    else:
        center = gdf.geometry.centroid.to_crs(epsg = 4326)

    # Calculate center coordinates
    lat = center.y.mean()
    lon = center.x.mean()

    # convert to WGS84 for mapping
    gdf = gdf.to_crs(epsg = 4326)

    # Create base map
    m = folium.Map(location = [lat, lon], zoom_start = 11)

    # Configure color.
    # Configure styling based on color_code parameter
    if color_code:
        unique_values = gdf[color_code].unique()
        num_colors = len(unique_values)

        # Use cmocean "phase" colormap.
        cmap = cmo.phase

        # Generate colors for discrete categories.
        colors = [mcolors.rgb2hex(
            cmap(i / max(num_colors-1, 1))
            )
            for i in range(num_colors)
            ]
        color_map = {val: colors[i] for i, val in enumerate(unique_values)}

        # Define style function with color coding.
        style_function = lambda x: {
            "fillColor": color_map.get(x['properties'][color_code], 'gray'),
            "color": "gray",
            "weight": 0.5,
            "fillOpacity": 0.35
        }

    else:
        # Define style function with single color
        style_function = lambda x: {
            "fillColor": "blue",
            "color": "black",
            "weight": 0.5,
            "fillOpacity": 0.35
        }
    
    # Add GeoJSON layer to map.
    folium.GeoJson(
        gdf,
        name = title,
        style_function = style_function,
        tooltip = folium.GeoJsonTooltip(fields = fields) if fields else None
    ).add_to(m)
    
    return m

#______________________________________________________________________________

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=e54228ee-ed5a-415e-97dc-d80557ea89cb' target="_blank">

Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>